In [2]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')
sys.path.append('../resnet_model')

In [4]:
from utils.general.read_files import read_from_json
from utils.general.dataset_variables import TripletSegmentationVariables

In [25]:
from collections import defaultdict
import json

def resolve_conflict_matching(conflict_json_path):
    with open(conflict_json_path, 'r') as f:
        issues_dict = json.load(f)

    resolved = {}
    explanation_dict = {}

    for vid, frames in issues_dict.items():
        for img_name, info in frames.items():
            if info['reason'] != 'conflict':
                continue
            
            cholec_instances = info['cholecinstanceseg_classes']
            cholec_triplets = info['triplet_classes']
            frame_id = img_name.replace('.json', '')

            class_triplets = defaultdict(list)
            for triplet in cholec_triplets:
                inst, verb, target = triplet.split(',')
                class_triplets[inst].append((verb, target))

            # 1️⃣ Instances & Triplets
            if cholec_instances and cholec_triplets:
                inst_counts = defaultdict(int)
                triplet_counts = defaultdict(int)

                for inst_class in cholec_instances:
                    inst_counts[inst_class] += 1
                    instance_id = inst_counts[inst_class]
                    key = f"{frame_id},{inst_class},{instance_id}"

                    if triplet_counts[inst_class] < len(class_triplets[inst_class]):
                        verb, target = class_triplets[inst_class][triplet_counts[inst_class]]
                        triplet_counts[inst_class] += 1
                        resolved[key] = f"{inst_class},{verb},{target}"
                        explanation_dict[key] = f"Matched by order: instance #{instance_id} to triplet #{triplet_counts[inst_class]}"
                    else:
                        resolved[key] = f"{inst_class},null_verb,null_target"
                        explanation_dict[key] = f"No triplet left for {inst_class} instance #{instance_id}  assigned to null_verb/null_target"

                # 2️⃣ Triplets with no matching class → artificial key
                for inst_class, triplets in class_triplets.items():
                    if inst_class not in inst_counts:
                        for i, (verb, target) in enumerate(triplets):
                            key = f"{frame_id},{inst_class},1000"
                            resolved[key] = f"{inst_class},{verb},{target}"
                            explanation_dict[key] = f"No instance for {inst_class} so assigned artificial key"

            # 3️⃣ Only Triplets
            elif cholec_triplets and not cholec_instances:
                for inst_class, triplets in class_triplets.items():
                    for i, (verb, target) in enumerate(triplets):
                        key = f"{frame_id},{inst_class},1000"
                        resolved[key] = f"{inst_class},{verb},{target}"
                        explanation_dict[key] = f"Only triplet found for {inst_class} so assigned artificial key"

            # 4️⃣ Only Instances
            elif cholec_instances and not cholec_triplets:
                inst_counts = defaultdict(int)
                for inst_class in cholec_instances:
                    inst_counts[inst_class] += 1
                    instance_id = inst_counts[inst_class]
                    key = f"{frame_id},{inst_class},{instance_id}"
                    resolved[key] = f"{inst_class},null_verb,null_target"
                    explanation_dict[key] = f"Only instance found for {inst_class} so assigned null_verb/null_target"

    return resolved, explanation_dict


In [26]:
conflict_json_path = 'outputs/error_dict_between_tripletsegmentation_v1.json'
resolved, explanation_dict =  resolve_conflict_matching(conflict_json_path)

In [27]:
explanation_dict

{'t50_VID01_000004,grasper,1000': 'Only triplet found for grasper so assigned artificial key',
 't50_VID01_000005,grasper,1': 'Only instance found for grasper so assigned null_verb/null_target',
 't50_VID01_000006,grasper,1': 'Only instance found for grasper so assigned null_verb/null_target',
 't50_VID01_000007,grasper,1': 'Only instance found for grasper so assigned null_verb/null_target',
 't50_VID01_000008,grasper,1': 'Only instance found for grasper so assigned null_verb/null_target',
 't50_VID01_000009,grasper,1': 'Only instance found for grasper so assigned null_verb/null_target',
 't50_VID01_000019,hook,1': 'No triplet left for hook instance #1  assigned to null_verb/null_target',
 't50_VID01_000019,grasper,1': 'Matched by order: instance #1 to triplet #1',
 't50_VID01_000020,grasper,1': 'Matched by order: instance #1 to triplet #1',
 't50_VID01_000020,hook,1000': 'No instance for hook so assigned artificial key',
 't50_VID01_000106,hook,1': 'Matched by order: instance #1 to tr

In [28]:
from utils.general.save_files import save_to_json

conflict_matched_file_path = 'outputs/conflict_matching_tripletsegmentation_v1.json'
conflict_matched_reason_path = 'outputs/conflict_matching_explanation_tripletsegmentation_v1.json'

save_to_json(data=resolved, json_file_path=conflict_matched_file_path)
save_to_json(data=explanation_dict, json_file_path=conflict_matched_reason_path)